# Paper 13: Attention Is All You Need
## Vaswani et al. (2017)

### The Transformer: Pure Attention Architecture

Revolutionary architecture that replaced RNNs with self-attention, enabling modern LLMs.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

## Scaled Dot-Product Attention

The fundamental building block:
$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

In [ ]:
def softmax(x, axis=-1):
    """Numerically stable softmax"""
    x_max = np.max(x, axis=axis, keepdims=True)
    exp_x = np.exp(x - x_max)
    return exp_x / np.sum(exp_x, axis=axis, keepdims=True)

def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    Scaled Dot-Product Attention
    
    Q: Queries (seq_len_q, d_k)
    K: Keys (seq_len_k, d_k)
    V: Values (seq_len_v, d_v)
    mask: Optional mask (seq_len_q, seq_len_k)
    """
    d_k = Q.shape[-1]
    
    # Compute attention scores
    scores = np.dot(Q, K.T) / np.sqrt(d_k)
    
    # Apply mask if provided (for causality or padding)
    if mask is not None:
        scores = scores + (mask * -1e9)
    
    # Softmax to get attention weights
    attention_weights = softmax(scores, axis=-1)
    
    # Weighted sum of values
    output = np.dot(attention_weights, V)
    
    return output, attention_weights

# Test scaled dot-product attention
seq_len = 5
d_model = 8

Q = np.random.randn(seq_len, d_model)
K = np.random.randn(seq_len, d_model)
V = np.random.randn(seq_len, d_model)

output, attn_weights = scaled_dot_product_attention(Q, K, V)

print(f"Attention output shape: {output.shape}")
print(f"Attention weights shape: {attn_weights.shape}")
print(f"Attention weights sum (should be 1): {attn_weights.sum(axis=1)}")

# Visualize attention pattern
plt.figure(figsize=(8, 6))
plt.imshow(attn_weights, cmap='viridis', aspect='auto')
plt.colorbar(label='Attention Weight')
plt.xlabel('Key Position')
plt.ylabel('Query Position')
plt.title('Attention Weights Matrix')
plt.show()

## Multi-Head Attention

Multiple attention "heads" attend to different aspects of the input:
$$\text{MultiHead}(Q,K,V) = \text{Concat}(head_1, ..., head_h)W^O$$

In [ ]:
class MultiHeadAttention:
    def __init__(self, d_model, num_heads):
        assert d_model % num_heads == 0
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        # Linear projections for Q, K, V for all heads (parallelized)
        self.W_q = np.random.randn(d_model, d_model) * 0.1
        self.W_k = np.random.randn(d_model, d_model) * 0.1
        self.W_v = np.random.randn(d_model, d_model) * 0.1
        
        # Output projection
        self.W_o = np.random.randn(d_model, d_model) * 0.1
    
    def split_heads(self, x):
        """Split into multiple heads: (seq_len, d_model) -> (num_heads, seq_len, d_k)"""
        seq_len = x.shape[0]
        x = x.reshape(seq_len, self.num_heads, self.d_k)
        return x.transpose(1, 0, 2)
    
    def combine_heads(self, x):
        """Combine heads: (num_heads, seq_len, d_k) -> (seq_len, d_model)"""
        seq_len = x.shape[1]
        x = x.transpose(1, 0, 2)
        return x.reshape(seq_len, self.d_model)
    
    def forward(self, Q, K, V, mask=None):
        """
        Multi-head attention forward pass
        
        Q, K, V: (seq_len, d_model)
        """
        # Linear projections
        Q = np.dot(Q, self.W_q.T)
        K = np.dot(K, self.W_k.T)
        V = np.dot(V, self.W_v.T)
        
        # Split into multiple heads
        Q = self.split_heads(Q)  # (num_heads, seq_len, d_k)
        K = self.split_heads(K)
        V = self.split_heads(V)
        
        # Apply attention to each head
        head_outputs = []
        self.attention_weights = []
        
        for i in range(self.num_heads):
            head_out, head_attn = scaled_dot_product_attention(
                Q[i], K[i], V[i], mask
            )
            head_outputs.append(head_out)
            self.attention_weights.append(head_attn)
        
        # Stack heads
        heads = np.stack(head_outputs, axis=0)  # (num_heads, seq_len, d_k)
        
        # Combine heads
        combined = self.combine_heads(heads)  # (seq_len, d_model)
        
        # Final linear projection
        output = np.dot(combined, self.W_o.T)
        
        return output

# Test multi-head attention
d_model = 64
num_heads = 8
seq_len = 10

mha = MultiHeadAttention(d_model, num_heads)

X = np.random.randn(seq_len, d_model)
output = mha.forward(X, X, X)  # Self-attention

print(f"\nMulti-Head Attention:")
print(f"Input shape: {X.shape}")
print(f"Output shape: {output.shape}")
print(f"Number of heads: {num_heads}")
print(f"Dimension per head: {mha.d_k}")

## Positional Encoding

Since Transformers have no recurrence, we add position information:
$$PE_{(pos, 2i)} = \sin(pos / 10000^{2i/d_{model}})$$
$$PE_{(pos, 2i+1)} = \cos(pos / 10000^{2i/d_{model}})$$

In [ ]:
def positional_encoding(seq_len, d_model):
    """
    Create sinusoidal positional encoding
    """
    pe = np.zeros((seq_len, d_model))
    
    position = np.arange(0, seq_len)[:, np.newaxis]
    div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))
    
    # Apply sin to even indices
    pe[:, 0::2] = np.sin(position * div_term)
    
    # Apply cos to odd indices
    pe[:, 1::2] = np.cos(position * div_term)
    
    return pe

# Generate positional encodings
seq_len = 50
d_model = 64
pe = positional_encoding(seq_len, d_model)

# Visualize positional encodings
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.imshow(pe.T, cmap='RdBu', aspect='auto')
plt.colorbar(label='Encoding Value')
plt.xlabel('Position')
plt.ylabel('Dimension')
plt.title('Positional Encoding (All Dimensions)')

plt.subplot(2, 1, 2)
# Plot first few dimensions
for i in [0, 1, 2, 3, 10, 20]:
    plt.plot(pe[:, i], label=f'Dim {i}')
plt.xlabel('Position')
plt.ylabel('Encoding Value')
plt.title('Positional Encoding (Selected Dimensions)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Positional encoding shape: {pe.shape}")
print(f"Different frequencies encode position at different scales")

## Feed-Forward Network

Applied to each position independently:
$$FFN(x) = \max(0, xW_1 + b_1)W_2 + b_2$$

In [ ]:
class FeedForward:
    def __init__(self, d_model, d_ff):
        self.W1 = np.random.randn(d_model, d_ff) * 0.1
        self.b1 = np.zeros(d_ff)
        self.W2 = np.random.randn(d_ff, d_model) * 0.1
        self.b2 = np.zeros(d_model)
    
    def forward(self, x):
        # First layer with ReLU
        hidden = np.maximum(0, np.dot(x, self.W1) + self.b1)
        
        # Second layer
        output = np.dot(hidden, self.W2) + self.b2
        
        return output

# Test feed-forward
d_model = 64
d_ff = 256  # Usually 4x larger

ff = FeedForward(d_model, d_ff)
x = np.random.randn(10, d_model)
output = ff.forward(x)

print(f"\nFeed-Forward Network:")
print(f"Input: {x.shape}")
print(f"Hidden: ({x.shape[0]}, {d_ff})")
print(f"Output: {output.shape}")

## Layer Normalization

Normalize across features (not batch like BatchNorm)

In [ ]:
class LayerNorm:
    def __init__(self, d_model, eps=1e-6):
        self.gamma = np.ones(d_model)
        self.beta = np.zeros(d_model)
        self.eps = eps
    
    def forward(self, x):
        mean = x.mean(axis=-1, keepdims=True)
        std = x.std(axis=-1, keepdims=True)
        
        normalized = (x - mean) / (std + self.eps)
        output = self.gamma * normalized + self.beta
        
        return output

ln = LayerNorm(d_model)
x = np.random.randn(10, d_model) * 3 + 5  # Unnormalized
normalized = ln.forward(x)

print(f"\nLayer Normalization:")
print(f"Input mean: {x.mean():.4f}, std: {x.std():.4f}")
print(f"Output mean: {normalized.mean():.4f}, std: {normalized.std():.4f}")

## Complete Transformer Block

In [ ]:
class TransformerBlock:
    def __init__(self, d_model, num_heads, d_ff):
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.norm1 = LayerNorm(d_model)
        self.ff = FeedForward(d_model, d_ff)
        self.norm2 = LayerNorm(d_model)
    
    def forward(self, x, mask=None):
        # Multi-head attention with residual connection
        attn_output = self.attention.forward(x, x, x, mask)
        x = self.norm1.forward(x + attn_output)
        
        # Feed-forward with residual connection
        ff_output = self.ff.forward(x)
        x = self.norm2.forward(x + ff_output)
        
        return x

# Test transformer block
block = TransformerBlock(d_model=64, num_heads=8, d_ff=256)
x = np.random.randn(10, 64)
output = block.forward(x)

print(f"\nTransformer Block:")
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"\nBlock contains:")
print(f"  1. Multi-Head Self-Attention")
print(f"  2. Layer Normalization")
print(f"  3. Feed-Forward Network")
print(f"  4. Residual Connections")

## Visualize Multi-Head Attention Patterns

In [ ]:
# Create attention with interpretable input
seq_len = 8
d_model = 64
num_heads = 4

mha = MultiHeadAttention(d_model, num_heads)
X = np.random.randn(seq_len, d_model)
output = mha.forward(X, X, X)

# Plot attention patterns for each head
fig, axes = plt.subplots(1, num_heads, figsize=(16, 4))

for i, ax in enumerate(axes):
    attn = mha.attention_weights[i]
    im = ax.imshow(attn, cmap='viridis', aspect='auto', vmin=0, vmax=1)
    ax.set_title(f'Head {i+1}')
    ax.set_xlabel('Key')
    ax.set_ylabel('Query')
    
plt.colorbar(im, ax=axes, label='Attention Weight', fraction=0.046, pad=0.04)
plt.suptitle('Multi-Head Attention Patterns', fontsize=14, y=1.05)
plt.tight_layout()
plt.show()

print("\nEach head learns to attend to different patterns!")
print("Different heads capture different relationships in the data.")

## Causal (Masked) Self-Attention for Autoregressive Models

In [ ]:
def create_causal_mask(seq_len):
    """Create mask to prevent attending to future positions"""
    mask = np.triu(np.ones((seq_len, seq_len)), k=1)
    return mask

# Test causal attention
seq_len = 8
causal_mask = create_causal_mask(seq_len)

Q = np.random.randn(seq_len, d_model)
K = np.random.randn(seq_len, d_model)
V = np.random.randn(seq_len, d_model)

# Without mask (bidirectional)
output_bi, attn_bi = scaled_dot_product_attention(Q, K, V)

# With causal mask (unidirectional)
output_causal, attn_causal = scaled_dot_product_attention(Q, K, V, mask=causal_mask)

# Visualize difference
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 5))

# Causal mask
ax1.imshow(causal_mask, cmap='Reds', aspect='auto')
ax1.set_title('Causal Mask\n(1 = masked/not allowed)')
ax1.set_xlabel('Key Position')
ax1.set_ylabel('Query Position')

# Bidirectional attention
im2 = ax2.imshow(attn_bi, cmap='viridis', aspect='auto', vmin=0, vmax=1)
ax2.set_title('Bidirectional Attention\n(can see future)')
ax2.set_xlabel('Key Position')
ax2.set_ylabel('Query Position')

# Causal attention
im3 = ax3.imshow(attn_causal, cmap='viridis', aspect='auto', vmin=0, vmax=1)
ax3.set_title('Causal Attention\n(cannot see future)')
ax3.set_xlabel('Key Position')
ax3.set_ylabel('Query Position')

plt.colorbar(im3, ax=[ax2, ax3], label='Attention Weight')
plt.tight_layout()
plt.show()

print("\nCausal masking is crucial for:")
print("  - Autoregressive generation (GPT, language models)")
print("  - Prevents information leakage from future tokens")
print("  - Each position can only attend to itself and previous positions")

## Key Takeaways

### Why "Attention Is All You Need"?
- **No recurrence**: Processes entire sequence in parallel
- **No convolution**: Pure attention mechanism
- **Scales better**: O(n²d) vs O(n) sequential operations in RNNs
- **Long-range dependencies**: Direct connections between any positions

### Core Components:
1. **Scaled Dot-Product Attention**: Efficient attention computation
2. **Multi-Head Attention**: Multiple representation subspaces
3. **Positional Encoding**: Inject position information
4. **Feed-Forward Networks**: Position-wise transformations
5. **Layer Normalization**: Stabilize training
6. **Residual Connections**: Enable deep networks

### Architecture Variants:
- **Encoder-Decoder**: Original Transformer (translation)
- **Encoder-only**: BERT (bidirectional understanding)
- **Decoder-only**: GPT (autoregressive generation)

### Advantages:
- Parallelizable training (unlike RNNs)
- Better long-range dependencies
- Interpretable attention patterns
- State-of-the-art on many tasks

### Impact:
- Foundation of modern NLP: GPT, BERT, T5, etc.
- Extended to vision: Vision Transformer (ViT)
- Multi-modal models: CLIP, Flamingo
- Enabled LLMs with billions of parameters